# Bank Statements Data Cleaning

This notebook processes and cleans a dataset of bank statements from 2025. The goal is to standardize formats, handle missing values, correct inconsistencies, and produce clean datasets for analysis.

## Step 1: Import Libraries and Load Data

We begin by importing the necessary libraries and loading the dataset.

In [13]:
import pandas as pd
import numpy as np
import charset_normalizer
from dateutil import parser as dtpr
import re
from fuzzywuzzy import fuzz, process

# Load the raw dataset
bank_data = pd.read_csv('../data/raw/bank_statements_2025.csv')

# Set seed for reproducibility
np.random.seed(42)

## Step 2: Initial Data Inspection

Let's examine the dataset to ensure it loaded correctly and identify any missing values (represented as NaN or None).

In [ ]:
bank_data.sample(10)

,Date,Description,Amount,Category,Account
0,2024-12-02,Convenience store purchase,389.03,GROCREIES,Credit Card
1,2025.02.23,Wages direct deposit,-50.57,Salaray,Credit Card
2,2024-10-15,Landlord payment,328.12,Rent,Savings
3,2025-02-22,Water utility payment,447.00,UTI,Credit Card
4,03 August 2025,Restaurant dining,400.18,Entervtainment,Credit Card


## Step 3: Assess Missing Data

We need to quantify missing values to understand the scale of the problem.

In [15]:
# Count missing values per column
missing_data_count = bank_data.isnull().sum()
missing_data_count

Date           3239
Description    3424
Amount         3324
Category       3316
Account        3497
dtype: int64

Let's calculate the percentage of missing data to assess its impact.

In [16]:
# Calculate total missing values and percentage
total_cells = np.prod(bank_data.shape)
total_missing = missing_data_count.sum()
percent_missing = round((total_missing / total_cells) * 100, 2)
print(f'{percent_missing}% of the data is missing')

5.01% of the data is missing


## Step 4: Verify File Encoding

Ensure the file's encoding is correct to avoid issues with text data.

In [17]:
# Check the first 15,000 bytes for encoding
with open('../data/raw/bank_statements_2025.csv', 'rb') as rawdata:
    result = charset_normalizer.detect(rawdata.read(15000))
print(result)

{'encoding': 'ascii', 'language': 'English', 'confidence': 1.0}


## Step 5: Standardize the Date Column

Convert dates to a consistent format (DD/MM/YY) and handle invalid entries.

In [18]:
def standardize_date(date_str):
    """
    Standardize date strings to 'DD/MM/YY' format.
    Args:
        date_str: Input date string.
    Returns:
        Formatted date string or NaN if invalid.
    """
    if pd.isna(date_str) or not isinstance(date_str, str) or date_str.strip() == '':
        return np.nan
    try:
        parsed_date = dtpr.parse(date_str)
        return parsed_date.strftime('%d/%m/%y')
    except (ValueError, TypeError):
        return np.nan

# Apply standardization and convert to datetime
bank_data['Date'] = bank_data['Date'].apply(standardize_date)
bank_data['Date'] = pd.to_datetime(bank_data['Date'], format='%d/%m/%y', errors='coerce')

# Handle remaining NaT with forward fill
bank_data['Date'] = bank_data['Date'].ffill()

## Step 6: Clean the Description Column

Handle missing values, standardize to string type, and ensure consistency.

In [19]:
# Fill missing values and convert to string
bank_data['Description'] = bank_data['Description'].fillna('Unspecified').astype('string')
bank_data['Description'] = bank_data['Description'].replace('nan', 'Unspecified')

# Trim whitespace
bank_data['Description'] = bank_data['Description'].str.strip()

# Display unique descriptions
bank_data['Description'].unique()

<StringArray>
[    'Convenience store purchase',           'Wages direct deposit',
               'Landlord payment',          'Water utility payment',
              'Restaurant dining',                     'Rental fee',
        'Concert ticket purchase',             'Phone bill payment',
           'Supermarket shopping',        'Food market transaction',
       'Electricity bill payment',         'Grocery store purchase',
             'Weekly grocery run',              'Employer paycheck',
          'Movie theater tickets',               'Freelance income',
               'Gas bill payment',                 'Salary payment',
                'Payroll deposit',        'Apartment lease payment',
           'Amusement park visit', 'Streaming service subscription',
                    'Unspecified',           'Monthly rent payment',
          'Internet service bill',          'Housing rent transfer']
Length: 26, dtype: string

## Step 7: Clean the Amount Column

Remove non-numeric characters, handle negative values, and interpolate missing values.

In [20]:
def clean_amount(amount):
    """
    Clean amount by removing non-numeric characters and rounding to 2 decimals.
    Args:
        amount: Input value.
    Returns:
        Float rounded to 2 decimals or NaN if invalid.
    """
    if pd.isna(amount) or amount == '':
        return np.nan
    amount_str = str(amount)
    cleaned = re.sub(r'[^\d.-]', '', amount_str)
    cleaned = re.sub(r'\.+', '.', cleaned)
    if cleaned.count('-') > 1:
        cleaned = '-' + cleaned.replace('-', '')
    try:
        return round(float(cleaned), 2)
    except ValueError:
        return np.nan

# Apply cleaning and interpolate missing values
bank_data['Amount'] = bank_data['Amount'].apply(clean_amount)
bank_data['Amount'] = bank_data['Amount'].interpolate(method='linear').round(2)

# Display cleaned amounts
bank_data['Amount']

0        389.03
1        -50.57
2        328.12
3        447.00
4        400.18
          ...  
67102   -322.76
67103   -278.45
67104   -332.44
67105    245.60
67106    -83.33
Name: Amount, Length: 67107, dtype: float64

## Step 8: Standardize the Category Column

Use fuzzy matching with an expanded synonym map to correct inconsistencies.

In [21]:
# Define valid categories and expanded synonym map
valid_categories = ['Groceries', 'Utilities', 'Entertainment', 'Salary', 'Rent', 'Unspecified']
synonym_map = {
    'Groceries': ['food', 'grocery', 'grocreies', 'supermarket', 'market'],
    'Utilities': ['bills', 'uti', 'utility', 'gas', 'electricity', 'water', 'phone', 'internet'],
    'Entertainment': ['fun', 'entervtainment', 'dining', 'movie', 'concert', 'streaming'],
    'Salary': ['income', 'salaray', 'wages', 'payroll', 'paycheck', 'freelance'],
    'Rent': ['housing', 'landlord', 'lease', 'rent']
}

# Create reverse synonym mapping
synonym_to_category = {syn.lower(): cat for cat, syn_list in synonym_map.items() for syn in syn_list}

def clean_category(category, threshold=80):
    """
    Standardize category using fuzzy matching and synonyms.
    Args:
        category: Input category string.
        threshold: Fuzzy matching score threshold.
    Returns:
        Standardized category or 'Unspecified' if no match.
    """
    if pd.isna(category) or not isinstance(category, str) or category.strip() == '':
        return 'Unspecified'
    category_lower = category.lower().strip()
    if category_lower in synonym_to_category:
        return synonym_to_category[category_lower]
    match = process.extractOne(category_lower, valid_categories, scorer=fuzz.ratio)
    return match[0] if match and match[1] >= threshold else 'Unspecified'

# Apply cleaning
bank_data['Category'] = bank_data['Category'].fillna('Unspecified').apply(clean_category)

## Step 9: Clean the Account Column

Ensure consistency by handling missing values and standardizing the format.

In [22]:
# Fill missing values and standardize
bank_data['Account'] = bank_data['Account'].fillna('Unspecified').astype(str)
bank_data['Account'] = bank_data['Account'].replace('nan', 'Unspecified').str.strip()

## Step 10: Export Cleaned Datasets

Save three versions of the cleaned data:
- With leeway (all data preserved).
- No leeway (drop rows with any remaining NaNs).
- Sorted by date (no leeway).

In [23]:
def perfect_cleanup(df):
    """
    Create a strict version of the dataset by dropping rows with any NaNs,
    except for Amount, which is interpolated.
    Args:
        df: Input DataFrame.
    Returns:
        Cleaned DataFrame with no missing values.
    """
    df_clean = df.copy()
    df_clean['Amount'] = df_clean['Amount'].interpolate(method='linear').round(2)
    df_clean = df_clean.dropna()
    return df_clean

# Export with leeway
bank_data.to_csv('../data/cleaned/bank_data_with_leeway.csv', index=False)

# Export no leeway
bank_data_improved = perfect_cleanup(bank_data)
bank_data_improved.to_csv('../data/cleaned/bank_data_no_leeway.csv', index=False)

# Export sorted by date (no leeway)
sorted_bank_data = bank_data_improved.sort_values(by='Date').reset_index(drop=True)
sorted_bank_data.to_csv('../data/cleaned/bank_data_sorted_nl.csv', index=False)